In [22]:
from pyannote.audio import Pipeline
from transformers import pipeline as transcribe_pipeline
import torchaudio
from huggingface_hub import login
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
from pydub import AudioSegment
import librosa

from config import HF_TOKEN

In [3]:
login(HF_TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/michael/.cache/huggingface/token
Login successful


In [4]:
# Initialize pyannote pipeline for speaker diarization
print("loading pyannote pipeline")
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")

# Initialize Whisper model for ASR (automatic speech recognition)
print("loading whisper model")
whisper_model = transcribe_pipeline("automatic-speech-recognition", model="openai/whisper-base")

# Path to your audio file
audio_file = "09-07-2017 audio_6 APCNV-2016-565.mp3"

# Perform diarization
print("performing diarization")
diarization = diarization_pipeline(audio_file)


loading pyannote pipeline


loading whisper model


performing diarization


In [10]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
# model_id = "openai/whisper-large-v3"
model_id = "openai/whisper-base"


model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)